In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

## Use Beautiful Soup to Get Match & Team Details (Upcoming Only)

In [2]:
url = "https://www.oddsportal.com/rugby-union/world/super-rugby/"
html = urlopen(url, timeout = 5)

In [3]:
soup = BeautifulSoup(html, 'lxml')

In [4]:
soup.find_all('tr',{'xeid':True})

[<tr class="odd" xeid="AFufydsq"><td class="table-time datet t1558078500-1-1-0-0"></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/hurricanes-jaguares-AFufydsq/">Hurricanes - Jaguares</a></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr xeid="80bczGdk"><td class="table-time datet t1558086300-1-1-0-0"></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/rebels-blue-bulls-80bczGdk/">Rebels - Bulls</a></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr class="odd" xeid="jPm2ZZBe"><td class="table-time datet t1558164900-1-1-0-0"></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/blues-waikato-chiefs-jPm2ZZBe/">Blues - C

In [5]:
games = soup.find_all(attrs={'class':'name table-participant'})

In [6]:
home_team = []
away_team = []

In [7]:
for game in games:
    teams = str(game).split('>')[2].split('<')[0].split(' - ')
    home_team.append(teams[0])
    away_team.append(teams[1])

## Use REST Service for Odds

In [8]:
import json
import requests
import objectpath

In [9]:
s = requests.session()
request_headers = {
    'Referer': 'https://www.oddsportal.com/rugby-union/world/super-rugby/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}

In [10]:
get_odds = s.get('https://fb.oddsportal.com/ajax-sport-country-tournament/8/z36c0gcb/X0/1',
                  headers = request_headers)

In [11]:
text_odds = get_odds.text[72:-2]

In [12]:
json_odds = json.loads(text_odds)

In [13]:
json_tree = objectpath.Tree(json_odds['d'])

In [14]:
odds = tuple(json_tree.execute('$..avg'))
# odds all appear twice for some reason, so we take just half of them
odds = odds[:np.int(len(odds)/2)]

In [15]:
one_list = []
x_list = []
two_list = []

In [16]:
# take every 3 rows and put them into 1 row
for i in range(len(odds)):
    if i%3==0:
        one_list.append(odds[i])
    if i%3==1:
        x_list.append(odds[i])
    if i%3==2:
        two_list.append(odds[i])

In [18]:
# concatenate info into dataframe
df = pd.DataFrame({'Home Team':home_team,
                    'Away Team':away_team,
                    'Home Odds':one_list[:len(games)],
                    'Draw Odds':x_list[:len(games)],
                    'Away Odds':two_list[:len(games)]})

In [20]:
df.head(len(games))

,Home Team,Away Team,Home Odds,Draw Odds,Away Odds
0,Hurricanes,Jaguares,1.21,28.27,4.40
1,Rebels,Bulls,1.39,23.23,3.03
2,Blues,Chiefs,1.47,21.64,2.73
3,Reds,Waratahs,1.96,19.99,1.87
4,Lions,Highlanders,1.87,20.02,1.97
5,Stormers,Crusaders,4.62,27.98,1.19
